## Build a RAG System with Langchain and ChromaDB

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# Langchain Imports

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# vector stores
from langchain_community.vectorstores import Chroma

# Utility imports
import numpy as np
from typing import List

In [7]:
sample_docs = [
    """
    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision trees, support vector machines, neural networks, and clustering techniques. As the field continues to evolve, advancements in deep learning and neural networks are driving significant progress in areas such as computer vision, natural language processing, and autonomous systems.
    """,

    """
    Natural Language Processing (NLP)

    Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. It involves the development of algorithms and models that enable machines to understand, interpret, and generate human language in a way that is meaningful. NLP encompasses various tasks such as text classification, sentiment analysis, named entity recognition, machine translation, and speech recognition. Techniques used in NLP include tokenization, part-of-speech tagging, syntactic parsing, and semantic analysis. Recent advancements in NLP have been driven by deep learning models such as transformers, which have significantly improved the performance of language models like BERT and GPT. Applications of NLP are widespread, including chatbots, virtual assistants, automated customer support, and content recommendation systems. As NLP continues to advance, it plays a crucial role in enhancing human-computer interaction and enabling more natural communication with technology.
    """,
    """
    Deep Learning and Neural Networks

    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of models that can automatically learn hierarchical representations of data. Key architectures in deep learning include convolutional neural networks (CNNs) for image processing, recurrent neural networks (RNNs) for sequential data, and transformers for language modeling. Training deep learning models typically requires large datasets and significant computational resources, often utilizing GPUs for efficient processing. The success of deep learning has led to breakthroughs in applications such as autonomous vehicles, medical image analysis, and real-time language translation. As research continues, deep learning is expected to drive further innovations in artificial intelligence and its applications across various domains.
    """
]

sample_docs

['\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision trees, support vector machines, neural networks, and clustering techniques. As the field continues to evolve, advancements in deep learning and neural networks 

In [8]:
### save sample documents to files
import tempfile
# temp_dir= tempfile.mkdtemp()

for i, doc in enumerate(sample_docs):
    # with open(os.path.join(temp_dir, f"doc_{i}.txt"), "w") as f:
    #     f.write(doc)
    with open(f"doc_{i}.txt", "w") as f:
        f.write(doc.strip())

print(f"Sample documents saved to root")

Sample documents saved to root


## Document Loading

In [9]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('.', glob='doc_*.txt', loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})

documents = loader.load()

print(f"Loaded {len(documents)} documents.")
print("First document content:")
print(documents[0].page_content[:200])  # print first 200 characters of the first document

Loaded 3 documents.
First document content:
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al


In [10]:
documents

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision trees, support vector machines, neural networks, and clustering techniques. As the field continues to evolve, 

## Document Splitting

In [11]:
# Initialize Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "] # Hierarchy of separators
)

chunks = text_splitter.split_documents(documents)

chunks

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision 

## Initialize ChromaDB Vector store and store the chunks

In [12]:
## Create a ChromaDB Vector Store

persist_directory = './chroma_db'

## Initialize ChromaDB with HuggingFace Embeddings
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    persist_directory=persist_directory,
    collection_name="rag_collection"
)

print(f"Vector store created with {vectorstore._collection.count()} vectors.")
print(f"Vectors persisted in directory: {persist_directory}")

Vector store created with 67 vectors.
Vectors persisted in directory: ./chroma_db


### Test Similarity search

In [13]:
query = "What are types of machine learning?"

similar_docs = vectorstore.similarity_search(query, k=3)

similar_docs

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsu

In [14]:
query = "What is NLP?"

similar_docs = vectorstore.similarity_search(query, k=3)

similar_docs

[Document(metadata={'source': 'doc_1.txt'}, page_content='Natural Language Processing (NLP)\n\n    Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. It involves the development of algorithms and models that enable machines to understand, interpret, and generate human language in a way that is meaningful. NLP encompasses various tasks such as text classification, sentiment analysis, named entity recognition, machine translation, and speech recognition. Techniques'),
 Document(metadata={'source': 'doc_1.txt'}, page_content='Natural Language Processing (NLP)\n\n    Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. It involves the development of algorithms and models that enable machines to understand, interpret, and generate human language in a way that is meaningful. NLP encompasses various tasks such as 

In [15]:
query = "What is deep learning?"

similar_docs = vectorstore.similarity_search(query, k=3)

similar_docs

[Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of models that'),
 Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields

### Advanced Similarity Search with Scores

In [16]:
result_scores= vectorstore.similarity_search_with_score(query, k=3)
result_scores

[(Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of models that'),
  0.5512487888336182),
 (Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revo

### Understanding Similarity Scores

The similarity score represents how closely related a document chunk is to your query, The scoring depends on the distance metric used:

ChromaDB by default uses L2 distance (Euclidean distance)
- Lower scores =  MORE similar (closer in vector space)
- Score of 0 =  identical vectors
- Typical range: 0 to 2 (but can be higher)

Cosine similarity (if configured):

- Higher scores =  MORE similar
- Range: -1 to 1 (1 being identical)

## Initialize LLM, RAG Chain, Prompt Tempate, Query the RAG system

In [17]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="mistral:latest")

In [18]:
test_response =  llm.invoke("What is Large Language Models?")

test_response

AIMessage(content=' Large Language Models (LLMs) are a type of artificial intelligence model designed to process and generate human-like text. They learn from vast amounts of text data, analyzing patterns and structures in the input to produce responses that mimic human conversation or writing style.\n\nThese models can perform various tasks such as answering questions, summarizing texts, translating languages, generating creative content like stories or poems, and even engaging in dialogues with users. They are trained on a diverse range of text sources, including books, websites, and social media, to ensure their responses are contextually relevant and coherent.\n\nExamples of large language models include me (I was created by Mistral AI), as well as models like ChatGPT (created by OpenAI) and BERT (developed by Google). These models have been a significant breakthrough in the field of natural language processing, enabling more human-like interactions between humans and AI.', additio

In [19]:
# Alternative waqy to initialize chat models

from langchain.chat_models.base import init_chat_model

chat_model = init_chat_model("ollama:mistral:latest")
#llm = init_chat_model("groq:")

chat_model

ChatOllama(model='mistral:latest')

In [20]:
chat_model.invoke("Explain the concept of reinforcement learning.")

AIMessage(content=' Reinforcement Learning (RL) is a type of machine learning where an agent learns to make decisions by taking actions in an environment to achieve a goal. The agent interacts with the environment by performing actions and receives rewards or punishments (referred to as "reinforcements") based on the outcomes of these actions.\n\nThe main objective of reinforcement learning is for the agent to learn a policy, which is a strategy that maps states of the environment to actions that maximize the cumulative reward over time. The agent\'s goal is to learn the optimal policy that will help it make the best decisions possible in the given environment.\n\nHere are some key components of reinforcement learning:\n\n1. Agent: An entity that interacts with an environment and makes decisions based on its observations and learned policies.\n2. Environment: The environment is where the agent performs actions, and changes its state as a result of these actions. The environment also pr

## Modern RAG Chain

In [21]:
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain

In [22]:
## Convert vectorstore to a retriever

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D8887EC410>, search_kwargs={'k': 3})

In [23]:
##Create Prompt Template
system_prompt = """You are an assistant for question-answering tasks.
Use the context provided to answer the question as accurately as possible.
If the context does not contain the answer, respond with 'I don't know'.
Use three sentences maximum to answer the question and keep it concise.

Context: {context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [24]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [25]:
## Create Documents Chain
documents_chain = create_stuff_documents_chain(
    llm=chat_model,
    prompt=prompt
)

documents_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOllama(model='mistral:latest')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_document

This chain:

- Takes retrieved documents
- "Stuffs" them into prompt's {context} placeholder.
- Sends the complete prompt to the LLM.
- Returns the LLM's response.

In [26]:
### Create the Final RAG Chain
rag_chain = create_retrieval_chain(retriever,documents_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D8887EC410>, search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the context provided to answer the question as accurately as possible

Using this create_retrieval_chain we are able to another retreval layer to our stuff document chain which uses the retriever to fetch relavant documents from our vectorstore and sends it to the stuff chain which sends it to the LLM.

In [27]:
response = rag_chain.invoke({"input": "Explain the different types of machine learning."})

response

{'input': 'Explain the different types of machine learning.',
 'context': [Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
  Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instru

In [28]:
response["answer"]

' Machine Learning can be categorized into three main types:\n\n1. Supervised Learning: This type uses labeled data to train models, where both input data and desired outputs are provided during training. Examples include linear regression and support vector machines (SVM).\n\n2. Unsupervised Learning: Unlike supervised learning, unsupervised learning deals with data that has no labels. The goal is for the algorithm to find patterns or structure in the data on its own, such as clustering similar data points together. Examples include k-means clustering and principal component analysis (PCA).\n\n3. Reinforcement Learning: This type involves an agent interacting with an environment to learn optimal actions based on rewards or punishments. The goal is for the agent to learn a policy that maximizes its cumulative reward over time, such as in game playing algorithms like AlphaGo or Deep Q-Network (DQN).'

In [29]:
# Function to query the modern RAG system

def query_rag_modern(question: str) -> str:
    print(f"Question: {question}")
    print("-"*50)

    # Using create_retrieval_chain approach
    result = rag_chain.invoke({"input": question})

    print(f"Answer: {result}")
    print("\nRetrieved Context:")
    for i, doc in enumerate(result['context']):
        print(f"\n ----- Source {i+1} -----")
        print(doc.page_content[:200]+"...")

    return result['answer']

test_questions = [
    "What are three types of machine learning?",
    "What is Deep Learning and how does it relate to Neural Networks?",
    "What are CNNs used for?"
]

for question in test_questions:
    query_rag_modern(question)
    print("\n"+"="*80+"\n")

Question: What are three types of machine learning?
--------------------------------------------------
Answer: {'input': 'What are three types of machine learning?', 'context': [Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'), Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algo

# Creating Rag Chain Alternative - Using LCEL

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


In [31]:
#Create a custom Prompt
custom_prompt = ChatPromptTemplate.from_template(
    """Use the context provided to answer the question as accurately as possible.
If the context does not contain the answer, respond with 'I don't know'.
Use three sentences maximum to answer the question and keep it concise.

Context: {context}

Question: {question}

Answer:"""
)

custom_prompt


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])

In [32]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D8887EC410>, search_kwargs={'k': 3})

In [33]:
## Format the output documents for the prompt
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in docs])

In [34]:
## Build the RAG Chain using LCEL
rag_chain_lcel = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()  # we use runnable passthrough to accept question input during runtime
    }
    | custom_prompt
    | chat_model
    | StrOutputParser()
)

rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D8887EC410>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])
| ChatOllama(model='mistral:latest')
| StrOutputParser()

In [35]:
response = rag_chain_lcel.invoke("Explain the different types of machine learning.")
response

' Machine Learning consists of three main types:\n1. Supervised Learning: It trains models using labeled data for specific tasks.\n2. Unsupervised Learning: It learns patterns from unlabeled data without explicit instructions.\n3. Reinforcement Learning: An agent learns to make decisions by interacting with an environment, getting rewards or penalties for each action.'

In [36]:
context = retriever.invoke("Explain the different types of machine learning.")
context

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsu

In [37]:
# Query using the LCEL RAG Chain

def query_rag_lcel(question: str) -> str:
    print(f"Question: {question}")
    print("-"*50)

    # Using create_retrieval_chain approach
    result = rag_chain_lcel.invoke(question)

    print(f"Answer: {result}")
    
    docs = retriever.invoke(question)
    print("\nSource Context:")

    for i, doc in enumerate(docs):
        print(f"\n ----- Source {i+1} -----")
        print(doc.page_content[:200]+"...")


test_questions = [
    "What are three types of machine learning?",
    "What is Deep Learning and how does it relate to Neural Networks?",
    "What are CNNs used for?"
]


In [38]:
for question in test_questions:
    query_rag_lcel(question)
    print("\n"+"="*80+"\n")

Question: What are three types of machine learning?
--------------------------------------------------
Answer:  Three types of Machine Learning are Supervised Learning, Unsupervised Learning, and Reinforcement Learning.

Source Context:

 ----- Source 1 -----
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al...

 ----- Source 2 -----
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al...

 ----- Source 3 -----
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al...


Question: What is Deep Learning and how does it relate to Neural Networks?
-----

# Adding new documents to existing vector store

In [39]:
vectorstore

In [40]:
# Adding new documents to existing vector store
new_document = """
Convolutional Neural Networks (CNNs) in detail

Convolutional Neural Networks (CNNs) are a class of deep learning models specifically designed for processing structured grid data, such as images. They utilize convolutional layers to automatically and adaptively learn spatial hierarchies of features from input data. CNNs are widely used in computer vision tasks, including image classification, object detection, and image segmentation, due to their ability to capture local patterns and reduce the number of parameters compared to fully connected networks. 

They have proven to be highly effective in various applications, such as facial recognition, medical image analysis, and autonomous driving, making them a fundamental component of modern AI systems.
"""

In [41]:
chunks

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision 

In [42]:
from langchain_core.documents import Document

new_doc = Document(page_content=new_document,
                   metadata={"source": "manual_addition", "topic": "CNNs"})

In [43]:
new_doc

Document(metadata={'source': 'manual_addition', 'topic': 'CNNs'}, page_content='\nConvolutional Neural Networks (CNNs) in detail\n\nConvolutional Neural Networks (CNNs) are a class of deep learning models specifically designed for processing structured grid data, such as images. They utilize convolutional layers to automatically and adaptively learn spatial hierarchies of features from input data. CNNs are widely used in computer vision tasks, including image classification, object detection, and image segmentation, due to their ability to capture local patterns and reduce the number of parameters compared to fully connected networks. \n\nThey have proven to be highly effective in various applications, such as facial recognition, medical image analysis, and autonomous driving, making them a fundamental component of modern AI systems.\n')

In [44]:
new_chunks = text_splitter.split_documents([new_doc])

In [45]:
# Add new chunks to existing vector store
vectorstore.add_documents(new_chunks)

['baba4f86-f8d2-4291-8aa6-9c1b9593f16f',
 '30becf7c-751b-436a-a1c5-a003680fdc9b']

In [46]:
print(f"Added {len(new_chunks)} new chunks to the vector store.")
print(f"Updated vector store now has {vectorstore._collection.count()} vectors.")

Added 2 new chunks to the vector store.
Updated vector store now has 69 vectors.


In [47]:
#Query after adding new documents

query = "What are Convolutional Neural Networks?"
query_rag_lcel(query)

Question: What are Convolutional Neural Networks?
--------------------------------------------------
Answer:  Convolutional Neural Networks (CNNs) are deep learning models designed for processing structured grid data like images. They use convolutional layers to automatically learn spatial hierarchies of features from input data, making them valuable in computer vision tasks such as image classification, object detection, and image segmentation.

Source Context:

 ----- Source 1 -----
Convolutional Neural Networks (CNNs) in detail

Convolutional Neural Networks (CNNs) are a class of deep learning models specifically designed for processing structured grid data, such as images. They...

 ----- Source 2 -----
Convolutional Neural Networks (CNNs) in detail

Convolutional Neural Networks (CNNs) are a class of deep learning models specifically designed for processing structured grid data, such as images. They...

 ----- Source 3 -----
Convolutional Neural Networks (CNNs) in detail

Convolut

# Advanced RAG Techniques- Conversational Memory

- create_history_aware_retriever: Makes retriever understand conversation context
- MessagesPlaceholder: Placeholder for chat history in prompts
- HumanMessage, AIMessage: Structured Message types for conversation history

In [48]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [54]:
# create a prompt with chat history placeholder

contextualized_q_system_prompt = """Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return the question as is.
"""

contextualized_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualized_q_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

In [55]:
## create a history aware retriever
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualized_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D8887EC410>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, T

In [56]:
# Create a new document chain with history

from openai import conversations


qa_system_prompt = """You are an assistant for question-answering tasks.
Use the context provided to answer the question as accurately as possible.
If the context does not contain the answer, respond with 'I don't know'.
Use three sentences maximum to answer the question and keep it concise.

Context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

queston_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create conversational RAG chain
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever,
    queston_answer_chain
)

print("Conversational RAG Chain created.")

Conversational RAG Chain created.


In [59]:
chat_history = []
#First Query
response1 = conversational_rag_chain.invoke({
    "input": "What is machine learning?",
    "chat_history": chat_history
})

print(f"Question: What is machine learning?")
print(f"Answer: {response1['answer']}")

Question: What is machine learning?
Answer:  Machine Learning is a subfield of Artificial Intelligence that enables computers to learn from data, enabling them to perform specific tasks without being explicitly programmed. It utilizes various algorithms and statistical models to make decisions based on data. The main types include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data for training models.


In [62]:
chat_history

chat_history.extend([HumanMessage(content="What is machine learning?"),
                    AIMessage(content=response1['answer'])])

In [63]:
# Follow-up Query
response2 = conversational_rag_chain.invoke({
    "input": "What are its main types?",
    "chat_history": chat_history
})

print(f"Question: What are its main types?")
print(f"Answer: {response2['answer']}")

response2

Question: What are its main types?
Answer:  Machine Learning's main types are Supervised Learning, Unsupervised Learning, and Reinforcement Learning. Supervised Learning utilizes labeled data for training models, while Unsupervised Learning involves finding patterns in unlabeled data. Reinforcement Learning is a method that trains an agent to make decisions by rewarding desired outcomes.


{'input': 'What are its main types?',
 'chat_history': [HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=' Machine Learning is a subfield of Artificial Intelligence that enables computers to learn from data, enabling them to perform specific tasks without being explicitly programmed. It utilizes various algorithms and statistical models to make decisions based on data. The main types include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data for training models.', additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without 